# Service optimization

In [ ]:
import geopandas as gpd

blocks = gpd.read_parquet('./../../data/blocks.parquet')
columns = [c for c in blocks.columns if 'capacity_' in c]
blocks = blocks[['geometry', 'population', 'site_area', *columns]].copy()
blocks.head(3)

In [ ]:
import pandas as pd

accessibility_matrix = pd.read_pickle('./../../data/accessibility_matrix.pickle')
accessibility_matrix.head()

### Select ID of optimized blocks

In [ ]:
from blocksnet.enums import LandUse
blocks_lu = {
    6954 : LandUse.BUSINESS
}

### Service distribution

In [ ]:
basic_services = {
    "kindergarten": 0.2,
    "school": 0.2,
    "pharmacy": 0.2,
    "polyclinics": 0.3,
    "convenience": 0.2,   
    "cafe": 0.15,
    "playground": 0.2,
    "post": 0.1,
    "hairdresser": 0.1
}
advanced_services = {
    "hospital": 0.5,
    "supermarket": 0.4,
    "restaurant": 0.3,
    "park": 0.8,
    "bank": 0.3,
    "fuel": 0.25,
    "police": 0.2,
    "cinema": 0.2,
    "market": 0.2,
    "religion": 0.5,
    "multifunctional_center": 0.1
}
comfort_services = {
    "mall": 0.3,   
    "swimming_pool": 0.4,
    "theatre": 0.2,
    "museum": 0.2,
    "university": 0.2,
    "sanatorium": 0.2,
}


def get_service_weight(service_name):
    if service_name in basic_services:
        return 0.5714 * basic_services[service_name] / sum(basic_services.values())
    elif service_name in advanced_services:
        return 0.2857 * advanced_services[service_name] / sum(advanced_services.values())
    elif service_name in comfort_services:
        return 0.1429 * comfort_services[service_name] / sum(comfort_services.values())
    return 0

### Service types and weights

In [ ]:
chosen_service_types = set(basic_services) | set(advanced_services) | set(comfort_services)
service_weights = {service_type: get_service_weight(service_type) for service_type in chosen_service_types if f'capacity_{service_type}' in blocks.columns}

### Initialize and run optimizer

In [ ]:
import csv
import numpy as np
def save_provisions_capacity(facade, var_adapter):
    start_prov = facade.start_provisions

    max_capacities = facade.get_max_capacities(1536)

    with open('./results/capacity_provisions_data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Service type", "Capacity required", "Total capacity of units", "Demand left (start)", "Demand left (after opt.)", "Opt. provisions", "Start provisions"])  # Write header row
        for st, max_cap in max_capacities.items():
            vars = var_adapter(np.zeros(facade.num_params))
            tot_cap = 0
            max_cap *= 1.2
            for var in vars:
                if var.service_type == st:
                    var.count = int(np.ceil(max_cap / var.capacity))
                    tot_cap = var.total_capacity
                    break
            vars_df = var_adapter._variables_to_df(vars)

            demand = facade._provision_adapter.get_start_provision_df(st)["demand_left"].sum()

            prov = facade._provision_adapter.calculate_provision(st, vars_df)

            demand_opt = facade._provision_adapter.get_last_provision_df(st)["demand_left"].sum()

            writer.writerow([st, max_cap, tot_cap, demand, demand_opt, prov, start_prov[st]])

In [ ]:
from blocksnet.optimization.services import (
    TPEOptimizer,
    WeightedObjective,
    WeightedConstraints,
    Facade,
    BlockSolution,
    GradientChooser,
)
from tqdm import tqdm

var_adapter = BlockSolution(blocks_lu)

facade = Facade(
    blocks_lu=blocks_lu,
    blocks_df=blocks,
    accessibility_matrix=accessibility_matrix,
    var_adapter=var_adapter,
)

for service_type, weight in tqdm(service_weights.items()):
    facade.add_service_type(service_type, weight, blocks.rename(columns={f'capacity_{service_type}': 'capacity'})[['capacity']])

objective = WeightedObjective(num_params=facade.num_params, facade=facade, weights=service_weights, max_evals=50)

constraints = WeightedConstraints(num_params=facade.num_params, facade=facade)

tpe_optimizer = TPEOptimizer(
    objective=objective, constraints=constraints, vars_chooser=GradientChooser(facade, facade.num_params, 5)
)

best_x, best_val, perc, func_evals = tpe_optimizer.run(max_runs=1000, timeout=60000, initial_runs_num=1)

Convert to services dataframe

In [ ]:
solution_df = facade.solution_to_services_df(best_x)
solution_df.to_csv(f'tpe_services.csv')
solution_df.head(5)

### Visualization

In [ ]:
import matplotlib.pyplot as plt


def plot_df(df):
    df_called_obj = df.loc[df["called_obj"] == True]

    plt.scatter(df_called_obj["func_evals"], df_called_obj["best_val"], c="red", s=5)

    df_called_obj = pd.concat(
        [pd.DataFrame([[-1, None, 0, 0, None, 0, False, 0]], columns=df_called_obj.columns), df_called_obj]
    )
    plt.plot(
        df_called_obj["func_evals"], df_called_obj["best_val"], label="Best Value (TPE)", color="blue", linewidth=1
    )
    # Добавление точек с цветом в зависимости от 'called_obj'
    # Настройка осей и заголовка
    plt.xlabel("Number of objective evaluation")
    plt.ylabel("Best Value")
    plt.title("Best Value Across Trials")
    plt.legend()
    plt.grid(True)

    # Отображение графика
    plt.show()

In [ ]:
import pandas as pd

tpe_df = pd.read_csv("tpe_trials.csv")
plot_df(tpe_df)